In [ ]:
import pathlib
import requests
import urllib
import boto3
import botocore
import dotenv
import os

In [ ]:
SCHEME = "https"
NETLOC = "data.linz.govt.nz"
PATH = "/services/query/v1/vector.json"
dotenv.load_dotenv()
KEY = os.environ.get('LINZ_API', None)

In [ ]:
params = {
        "key": KEY,
        "layer": 105448,
        "x": 171.99059591063107,
        "y": -41.78711137951029,
        "max_results": 3,
        "radius": 100000,
        "geometry": "true",
        "with_field_names": "true",
        "Accept-Encoding": "gzip"
    }

https://data.linz.govt.nz/services/query/v1/vector.json?key=b74b510da6cb4e23ad8e779f9bd1b366&layer=105448&x=171.99059591063107&y=-41.78711137951029&max_results=3&radius=10000&geometry=true&with_field_names=true

In [ ]:
data_url = urllib.parse.urlunparse((SCHEME, NETLOC, PATH, "", "", ""))

response = requests.get(data_url, params=params, stream=True)
response.raise_for_status()
json_response=response.json()

In [ ]:
requests.Request('POST', data_url, params=params).prepare().url

In [ ]:
json_response

In [ ]:
id_name=json_response['Datasets'][0]['Dataset']['alternateName']
id_name

# Issues with cql_filter bbox and correct lable 
try generated URLs in browser to check if valid or 404